In [119]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

data_frame = pd.read_excel('attaccanti_23.xlsx')


In [120]:
data_frame.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK',
       'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA',
       'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1',
       'Att 3rd.1', 'Att.1', 'Succ', 'Succ%', 'Carries', 'TotDist', '2CrdY',
       'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Won%'],
      dtype='object')

tolgo le colonne che non servono per la analisi dati quindi nome, squadra e posizione

In [121]:
data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad','CrdY', 'CrdR', 'Tkl', 'TklW', 'Mid 3rd',
       'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ',
       'Succ%', 'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won',
       'Lost', 'Won%'], inplace = True)

data_frame = data_frame[data_frame['Min']>=100]

STATS CORRELATIONS

In [122]:
fig = px.imshow(data_frame.corr())
fig.show()